# Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá | Semana 3 - Parte 1, 2 y 3

## Parte 1

### Importación de Librerías


In [21]:
import pandas as pd
import numpy as np

### Traemos y transformamos los datos de la URL de Wikipedia

In [22]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
df = pd.read_html(url)[0]
df.head(n=10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Se ignoran las celdas con municipios Not Assigned

In [24]:
df = df[df['Borough'] != 'Not assigned']
df.head(n=10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


### Transformación de valores

In [25]:
df['Neighbourhood'] = df.groupby('Postal Code')['Neighbourhood'].transform(lambda x: ', '.join(x))
df.head(n=10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Si una celda tiene un municipio, pero un vecindario No asignado, entonces el vecindario será el mismo que el municipio.

In [26]:
mode =df[df['Neighbourhood']!="Not assigned"]['Neighbourhood'].mode()[0]
df.head(n=10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


### Número de Filas

In [27]:
df.shape[0]

103

## Parte 2

### Cargamos los datos de latitud y longitud

In [35]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv")
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge de los dataframes

In [36]:
df_merged = pd.merge(df,df_coord, on='Postal Code')
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Parte 3

In [37]:
import json 
from geopy.geocoders import Nominatim 

import requests 

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing th

### Se buscan las coordenas de la ciudad de Toronto

In [39]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas son {}, {}.'.format(latitude, longitude))

Las coordenadas son 43.6534817, -79.3839347.


In [40]:
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [44]:
CLIENT_ID = '3NWC2RRSGI2QLUVM15DPPKHSLARATWRXMXJ42LLIFQAH4M3W' # your Foursquare ID
CLIENT_SECRET = '3RAD0IKREQCSEHXH0MVABB3TXK45Q0KU15GCWW0YAQ4KNZS3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [45]:
df_toronto.loc[0, 'Neighbourhood']


'Regent Park, Harbourfront'

In [47]:
address = 'TORONTO,CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas son {}, {}.'.format(latitude, longitude))

Las coordenadas son 43.6534817, -79.3839347.


In [49]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Peruvian .... OK!


In [53]:
import requests
LIMIT = 100

radius = 500

url = "https://api.foursquare.com/v3/places/search?query=Italian&ll=43.6534%2C-79.3839&radius=500"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3332FQkdWjIB9PRXRxCeARYgA/tiEVC7JXYRtKAYVn+U="
}

response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4d306dd82748b60c62b6dba0","categories":[{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":262,"geocodes":{"main":{"latitude":43.654458,"longitude":-79.381142},"roof":{"latitude":43.654458,"longitude":-79.381142}},"link":"/v3/places/4d306dd82748b60c62b6dba0","location":{"address":"220 Yonge St","address_extended":"Level 3","country":"CA","cross_street":"in Toronto Eaton Centre","formatted_address":"220 Yonge St (in Toronto Eaton Centre), Toronto ON M5B 2H1","locality":"Toronto","neighborhood":["Financial District"],"postcode":"M5B 2H1","region":"ON"},"name":"Trattoria Mercatto","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"4afcc52df964a520b82522e3","categories":[{"id":13049,"name":"Diner","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/diner_","suffix":".png"}},{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/im

In [54]:
data = {
"results":[{"fsq_id":"4d306dd82748b60c62b6dba0","categories":[{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":262,"geocodes":{"main":{"latitude":43.654458,"longitude":-79.381142},"roof":{"latitude":43.654458,"longitude":-79.381142}},"link":"/v3/places/4d306dd82748b60c62b6dba0","location":{"address":"220 Yonge St","address_extended":"Level 3","country":"CA","cross_street":"in Toronto Eaton Centre","formatted_address":"220 Yonge St (in Toronto Eaton Centre), Toronto ON M5B 2H1","locality":"Toronto","neighborhood":["Financial District"],"postcode":"M5B 2H1","region":"ON"},"name":"Trattoria Mercatto","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"4afcc52df964a520b82522e3","categories":[{"id":13049,"name":"Diner","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/diner_","suffix":".png"}},{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":455,"geocodes":{"main":{"latitude":43.657373,"longitude":-79.383522},"roof":{"latitude":43.657373,"longitude":-79.383522}},"link":"/v3/places/4afcc52df964a520b82522e3","location":{"address":"37 Elm St","country":"CA","cross_street":"at Bay St.","formatted_address":"37 Elm St (at Bay St.), Toronto ON M5G 1H1","locality":"Toronto","neighborhood":["Downtown Toronto"],"postcode":"M5G 1H1","region":"ON"},"name":"Donatello Restaurant","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"4c251062db519521621d2c3a","categories":[{"id":13064,"name":"Pizzeria","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/pizza_","suffix":".png"}},{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":495,"geocodes":{"main":{"latitude":43.652159,"longitude":-79.378017},"roof":{"latitude":43.652159,"longitude":-79.378017}},"link":"/v3/places/4c251062db519521621d2c3a","location":{"address":"106 Victoria St","country":"CA","formatted_address":"106 Victoria St, Toronto ON M5C 2B4","locality":"Toronto","neighborhood":["Downtown Toronto"],"postcode":"M5C 2B4","region":"ON"},"name":"La Bettola Di Terroni","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"4b846dd4f964a520dc3431e3","categories":[{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":348,"geocodes":{"main":{"latitude":43.650351,"longitude":-79.384368},"roof":{"latitude":43.650351,"longitude":-79.384368}},"link":"/v3/places/4b846dd4f964a520dc3431e3","location":{"address":"121 Richmond St W","country":"CA","cross_street":"at York St.","formatted_address":"121 Richmond St W (at York St.), Toronto ON M5H 2K1","locality":"Toronto","neighborhood":["Financial District"],"postcode":"M5H 2K1","region":"ON"},"name":"Little Anthony's","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"0486cbc8dea547dcadbaeeee","categories":[{"id":13064,"name":"Pizzeria","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/pizza_","suffix":".png"}},{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":305,"geocodes":{"main":{"latitude":43.653912,"longitude":-79.380166},"roof":{"latitude":43.653912,"longitude":-79.380166}},"link":"/v3/places/0486cbc8dea547dcadbaeeee","location":{"address":"218 Yonge St","country":"CA","formatted_address":"218 Yonge St, Toronto ON M5B 2H6","locality":"Toronto","neighborhood":["Downtown Toronto"],"postcode":"M5B 2H6","region":"ON"},"name":"Mrs. Vanellis Fresh Italian Food & Pizza","related_places":{}},{"fsq_id":"5979225fa87921424da2e994","categories":[{"id":13002,"name":"Bakery","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/bakery_","suffix":".png"}},{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":476,"geocodes":{"main":{"latitude":43.650923,"longitude":-79.378803},"roof":{"latitude":43.650923,"longitude":-79.378803}},"link":"/v3/places/5979225fa87921424da2e994","location":{"address":"132 Yonge St","country":"CA","cross_street":"Adelaide","formatted_address":"132 Yonge St (Adelaide), Toronto ON M5C 1X3","locality":"Toronto","neighborhood":["Downtown Toronto"],"postcode":"M5C 1X3","region":"ON"},"name":"Sud Forno","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"5615b6c4498e3c32c67ad78f","categories":[{"id":13064,"name":"Pizzeria","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/pizza_","suffix":".png"}}],"chains":[],"distance":466,"geocodes":{"main":{"latitude":43.656609,"longitude":-79.380128},"roof":{"latitude":43.656609,"longitude":-79.380128}},"link":"/v3/places/5615b6c4498e3c32c67ad78f","location":{"address":"10 Dundas St E","address_extended":"# 124","country":"CA","cross_street":"","formatted_address":"10 Dundas St E, Toronto ON M5B 2G9","locality":"Toronto","neighborhood":["Downtown"],"postcode":"M5B 2G9","region":"ON"},"name":"Blaze Pizza","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"59f81cc775eee449f376720b","categories":[{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":376,"geocodes":{"main":{"latitude":43.653485,"longitude":-79.388373},"roof":{"latitude":43.653485,"longitude":-79.388373}},"link":"/v3/places/59f81cc775eee449f376720b","location":{"address":"17 Yorkville Ave","country":"CA","cross_street":"","formatted_address":"17 Yorkville Ave, Toronto county ON M4W 1L1","locality":"Toronto county","postcode":"M4W 1L1","region":"ON"},"name":"Blu Ristorante","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"5e5594f7fc85940008917b87","categories":[{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":447,"geocodes":{"main":{"latitude":43.657249,"longitude":-79.38369},"roof":{"latitude":43.657249,"longitude":-79.38369}},"link":"/v3/places/5e5594f7fc85940008917b87","location":{"address":"37 Elm St","country":"CA","cross_street":"","formatted_address":"37 Elm St, Toronto ON M5G 1H1","locality":"Toronto","postcode":"M5G 1H1","region":"ON"},"name":"Donetello","related_places":{},"timezone":"America/Toronto"},{"fsq_id":"5d28b9a17cfbfe0023bc907d","categories":[{"id":13236,"name":"Italian Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/italian_","suffix":".png"}}],"chains":[],"distance":473,"geocodes":{"main":{"latitude":43.656306,"longitude":-79.388168},"roof":{"latitude":43.656306,"longitude":-79.388168}},"link":"/v3/places/5d28b9a17cfbfe0023bc907d","location":{"address":"525 University Ave","address_extended":"# 3","country":"CA","cross_street":"","formatted_address":"525 University Ave, Toronto ON M5G 2L3","locality":"Toronto","postcode":"M5G 2L3","region":"ON"},"name":"Pasta Mondo","related_places":{},"timezone":"America/Toronto"}],"context":{"geo_bounds":{"circle":{"center":{"latitude":43.6534,"longitude":-79.3839},"radius":500}}}
}


In [55]:
df_dataframe = []
from unicodedata import category
import pandas as pd
result = data['results']
for entry in result:
    name = entry["name"]
    category = entry["categories"][0]['name']
    geocode_dct = entry['geocodes']['main']
    lat = geocode_dct['latitude']
    lon = geocode_dct['longitude']

    df_dataframe.append({
        "name": name,
        "lat": lat,
        "lon": lon,
        "category": category
    })
df = pd.DataFrame(df_dataframe)
print(df)

                                       name        lat        lon  \
0                        Trattoria Mercatto  43.654458 -79.381142   
1                      Donatello Restaurant  43.657373 -79.383522   
2                     La Bettola Di Terroni  43.652159 -79.378017   
3                          Little Anthony's  43.650351 -79.384368   
4  Mrs. Vanellis Fresh Italian Food & Pizza  43.653912 -79.380166   
5                                 Sud Forno  43.650923 -79.378803   
6                               Blaze Pizza  43.656609 -79.380128   
7                            Blu Ristorante  43.653485 -79.388373   
8                                 Donetello  43.657249 -79.383690   
9                               Pasta Mondo  43.656306 -79.388168   

             category  
0          Restaurant  
1               Diner  
2            Pizzeria  
3  Italian Restaurant  
4            Pizzeria  
5              Bakery  
6            Pizzeria  
7  Italian Restaurant  
8  Italian Restaurant  


In [56]:
import pandas as pd
import requests
import json
# define the function
def getNearbyVenues(names, latitudes, longitudes, radius=500,limit=30):

    URL= "https://api.foursquare.com/v3/places/search?query=Italian&ll=43.6534%2C-79.3839&radius=500"
    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3332FQkdWjIB9PRXRxCeARYgA/tiEVC7JXYRtKAYVn+U="
    }

    df_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = URL.format(lat, lng, radius, LIMIT)
        results = requests.request("GET", url, headers=headers).json()
        
        for each_result in results['results']: # filter the result based on JSON identification
            result={}
            result['Neighborhood']=name
            result['Neighborhood Latitude']=lat
            result['Neighborhood Longitude']=lng
            result['Name']=each_result['name']
            result['Venue Latitude']=each_result['geocodes']['main']['latitude']
            result['Venue Longitude']=each_result['geocodes']['main']['longitude']
            result['Locality']=each_result['location']['locality']
            result['Category_Names']=[each_name['name'] for each_name in each_result['categories']]
            df_list.append(result.copy())
    return pd.DataFrame(df_list) # return dataframe

In [57]:
names=['test_locations']
latitudes = [60.172667]
longitudes = [24.932009]
radius = 500
LIMIT = 5
# call the function
df_result=getNearbyVenues(names,latitudes,longitudes)
df_result.head(n=10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Name,Venue Latitude,Venue Longitude,Locality,Category_Names
0,test_locations,60.172667,24.932009,Trattoria Mercatto,43.654458,-79.381142,Toronto,[Restaurant]
1,test_locations,60.172667,24.932009,Donatello Restaurant,43.657373,-79.383522,Toronto,"[Diner, Italian Restaurant]"
2,test_locations,60.172667,24.932009,La Bettola Di Terroni,43.652159,-79.378017,Toronto,"[Pizzeria, Italian Restaurant]"
3,test_locations,60.172667,24.932009,Little Anthony's,43.650351,-79.384368,Toronto,[Italian Restaurant]
4,test_locations,60.172667,24.932009,Mrs. Vanellis Fresh Italian Food & Pizza,43.653912,-79.380166,Toronto,"[Pizzeria, Italian Restaurant]"
5,test_locations,60.172667,24.932009,Sud Forno,43.650923,-79.378803,Toronto,"[Bakery, Restaurant]"
6,test_locations,60.172667,24.932009,Blaze Pizza,43.656609,-79.380128,Toronto,[Pizzeria]
7,test_locations,60.172667,24.932009,Blu Ristorante,43.653485,-79.388373,Toronto county,[Italian Restaurant]
8,test_locations,60.172667,24.932009,Donetello,43.657249,-79.383690,Toronto,[Italian Restaurant]
9,test_locations,60.172667,24.932009,Pasta Mondo,43.656306,-79.388168,Toronto,[Italian Restaurant]
